In [1]:
# import packages
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv) 

In [2]:
df=pd.read_csv("WOE_calc2.csv")

In [3]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991265 entries, 0 to 1991264
Data columns (total 71 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Unnamed: 0                   int64  
 1   loan_amnt                    int64  
 2   funded_amnt                  int64  
 3   funded_amnt_inv              float64
 4   int_rate                     float64
 5   installment                  float64
 6   grade                        object 
 7   home_ownership               object 
 8   annual_inc                   float64
 9   verification_status          object 
 10  loan_status                  object 
 11  pymnt_plan                   object 
 12  purpose                      object 
 13  dti                          float64
 14  delinq_2yrs                  float64
 15  inq_last_6mths               float64
 16  open_acc                     float64
 17  pub_rec                      float64
 18  revol_bal                    int64  
 19  

In [5]:
df1=df.drop(["Unnamed: 0","loan_status"],axis=1)

In [6]:
numericaln = []
categoricaln = []
datesn = []

for col in list(df1.columns):
    if df1[col].dtype == "float64" or df1[col].dtype == "int64":
        numericaln.append(col)
    elif df1[col].dtype == "object":
        categoricaln.append(col) 
    elif df1[col].dtype == "<M8[ns]":
        datesn.append(col)  

In [7]:
len(numericaln)

61

In [8]:
len(categoricaln)

8

In [9]:
final_iv, IV = data_vars(df1,df1.good_bad)

In [10]:
final_iv.head(10)

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,loan_amnt,1000.0,13125.0,995634,883149,0.887022,112485,0.112978,0.506833,0.452148,0.114171,0.011994
1,loan_amnt,13150.0,40000.0,995631,859337,0.863108,136294,0.136892,0.493167,0.547852,-0.105156,0.011994
2,funded_amnt,1000.0,13125.0,995636,883151,0.887022,112485,0.112978,0.506834,0.452148,0.114173,0.011994
3,funded_amnt,13150.0,40000.0,995629,859335,0.863108,136294,0.136892,0.493166,0.547852,-0.105159,0.011994
4,funded_amnt_inv,725.0,13125.0,995768,883259,0.887013,112509,0.112987,0.506896,0.452245,0.114082,0.011979
5,funded_amnt_inv,13150.0,40000.0,995497,859227,0.863114,136270,0.136886,0.493104,0.547755,-0.105108,0.011979
6,int_rate,5.31,6.99,170324,166117,0.975300,4207,0.024700,0.095333,0.016911,1.729439,0.457045
7,int_rate,7.02,7.97,166895,160108,0.959334,6787,0.040666,0.091885,0.027281,1.214336,0.457045
8,int_rate,7.99,9.44,168273,157199,0.934190,11074,0.065810,0.090215,0.044513,0.706409,0.457045
9,int_rate,9.49,10.49,161756,150905,0.932917,10851,0.067083,0.086603,0.043617,0.685890,0.457045


In [11]:
final_iv.shape

(385, 12)

In [12]:
final_iv_num = final_iv[final_iv['VAR_NAME'].isin(numericaln)]

In [13]:
final_iv_num.shape

(347, 12)

In [14]:
final_iv_cat = final_iv[final_iv['VAR_NAME'].isin(categoricaln)]

In [15]:
final_iv_cat.shape

(38, 12)

In [16]:
final_iv_sorted=final_iv.loc[:,["VAR_NAME","IV"]]

In [17]:
final_iv_sorted=final_iv_sorted.drop_duplicates().sort_values(by='IV')

In [ ]:
#final_iv_sorted_SP=final_iv_sorted[(final_iv_sorted["IV"]>=0.3) & (final_iv_sorted["IV"]<=0.7) ]

In [18]:
a=[]
b=[]
for i in numericaln:
    dfs=final_iv[final_iv["VAR_NAME"]==i]
    for index, row in dfs.iterrows():
        a.append(row['MIN_VALUE'])
        b.append(row['MAX_VALUE'])
    for (x,y) in zip(a,b):
        df1["{}_({}_{})".format(i,x,y)]=0
        df1.loc[(df1[i]>x)&(df1[i]<=y),"{}_({}_{})".format(i,x,y)]=1
    a.clear()
    b.clear()
df1=df1.drop(numericaln,axis=1)

C:\Users\saura\AppData\Local\Temp/ipykernel_20120/3027035165.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df1["{}_({}_{})".format(i,x,y)]=0


In [19]:
df1.head()

,grade,home_ownership,verification_status,pymnt_plan,purpose,initial_list_status,application_type,disbursement_method,loan_amnt_(1000.0_13125.0),loan_amnt_(13150.0_40000.0),...,total_bc_limit_(49810.0_66300.0),total_bc_limit_(66313.0_1569000.0),total_il_high_credit_limit_(0.0_34092.0),total_il_high_credit_limit_(34093.0_2118996.0),term_int_(36_36),term_int_(60_60),mths_since_issue_d_(0.0_29.0),mths_since_issue_d_(30.0_64.0),mths_since_earliest_cr_line_(25.0_198.0),mths_since_earliest_cr_line_(199.0_1017.0)
0,C,RENT,Not Verified,n,debt_consolidation,w,Individual,Cash,1,0,...,0,0,1,0,0,0,0,1,0,1
1,D,MORTGAGE,Source Verified,n,debt_consolidation,w,Individual,Cash,0,1,...,0,0,0,1,0,0,0,1,0,1
2,D,MORTGAGE,Source Verified,n,debt_consolidation,w,Individual,Cash,1,0,...,0,0,1,0,0,0,0,1,1,0
3,D,MORTGAGE,Source Verified,n,debt_consolidation,w,Individual,Cash,1,0,...,0,0,0,1,0,0,0,1,1,0
4,C,MORTGAGE,Not Verified,n,debt_consolidation,w,Individual,Cash,0,1,...,0,0,1,0,0,0,0,1,0,1


In [20]:
a=[]
for i in categoricaln:
    dfs=final_iv[final_iv["VAR_NAME"]==i]
    for index, row in dfs.iterrows():
        a.append(row['MIN_VALUE'])
    for x in a:
        df1["{}_{}".format(i,x)]=0
        df1.loc[(df1[i]==x),"{}_{}".format(i,x)]=1
    a.clear()
    b.clear()
df1=df1.drop(categoricaln,axis=1)

In [21]:
df1.head()

,loan_amnt_(1000.0_13125.0),loan_amnt_(13150.0_40000.0),funded_amnt_(1000.0_13125.0),funded_amnt_(13150.0_40000.0),funded_amnt_inv_(725.0_13125.0),funded_amnt_inv_(13150.0_40000.0),int_rate_(5.31_6.99),int_rate_(7.02_7.97),int_rate_(7.99_9.44),int_rate_(9.49_10.49),...,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,disbursement_method_Cash,disbursement_method_DirectPay
0,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
1,0,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
2,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
3,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
4,0,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0


In [22]:
df1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991265 entries, 0 to 1991264
Data columns (total 385 columns):
 #    Column                                          Dtype
---   ------                                          -----
 0    loan_amnt_(1000.0_13125.0)                      int64
 1    loan_amnt_(13150.0_40000.0)                     int64
 2    funded_amnt_(1000.0_13125.0)                    int64
 3    funded_amnt_(13150.0_40000.0)                   int64
 4    funded_amnt_inv_(725.0_13125.0)                 int64
 5    funded_amnt_inv_(13150.0_40000.0)               int64
 6    int_rate_(5.31_6.99)                            int64
 7    int_rate_(7.02_7.97)                            int64
 8    int_rate_(7.99_9.44)                            int64
 9    int_rate_(9.49_10.49)                           int64
 10   int_rate_(10.56_11.49)                          int64
 11   int_rate_(11.53_12.62)                          int64
 12   int_rate_(12.69_13.56)                  

In [23]:
df1["good_bad"]=df["good_bad"]

In [24]:
df1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991265 entries, 0 to 1991264
Data columns (total 386 columns):
 #    Column                                          Dtype
---   ------                                          -----
 0    loan_amnt_(1000.0_13125.0)                      int64
 1    loan_amnt_(13150.0_40000.0)                     int64
 2    funded_amnt_(1000.0_13125.0)                    int64
 3    funded_amnt_(13150.0_40000.0)                   int64
 4    funded_amnt_inv_(725.0_13125.0)                 int64
 5    funded_amnt_inv_(13150.0_40000.0)               int64
 6    int_rate_(5.31_6.99)                            int64
 7    int_rate_(7.02_7.97)                            int64
 8    int_rate_(7.99_9.44)                            int64
 9    int_rate_(9.49_10.49)                           int64
 10   int_rate_(10.56_11.49)                          int64
 11   int_rate_(11.53_12.62)                          int64
 12   int_rate_(12.69_13.56)                  

In [26]:
df1=df1.drop(["loan_amnt_(1000.0_13125.0)","loan_amnt_(13150.0_40000.0)"],axis=1)

In [29]:
#df1.to_csv("PD_train_data_2.csv")

In [28]:
#df1.to_pickle("PD_train_data_2.pkl")